## Imports

In [1]:
import pandas as pd
import numpy as np

## Energy Consumption, Production, PPP, Population
---
Retrieved from [U.S. Energy Information Administration](https://www.eia.gov/international/data/world).

### Energy Consumption and Production by Source
---
Data on consumption retrieved as multiple tables in one Excel sheet. First, the data was manipulated into separate sheets by source. The data was then melted from wide- to long-format, using country-year combinations as keys.

Energy consumption and production measured in quadrillion British thermal units (`Btu`, amount of heat required to raise one pound of water by one degree Fahrenheit).

In [2]:
# energy consumption and energy production into one dataframe

#creating dataframe of country-year combinations to use as keys for merging
e_cons = pd.read_excel('../data/energy_consumption.xlsx')
main_df = pd.melt(e_cons,
                 id_vars=['country'],
                 value_vars=e_cons.columns[1:],
                 var_name=['year'])[['country','year']]

# files, sheets for energy consumption/production
e_files = ['energy_production', 'energy_consumption']
e_sheets = ['total','coal','natural_gas','petrolium_and_other_liquids',
            'nuclear', 'nuclear_renewables_and_other','renewables_and_other']

for file in e_files: # reading in files
    for sheet in e_sheets: # reading in each sheet as temporary dataframe
        
        # reading in each sheet in both files as temporary dataframe
        temp_df = pd.read_excel('../data/'+file+'.xlsx',sheet_name=sheet)
        
        # melting temp_df to have columns ['country','year'] as keys and
        # activity-source as values
        temp_df = pd.melt(temp_df,
                          id_vars = ['country'],
                          value_vars = temp_df.columns[1:],
                          var_name = ['year'],
                          value_name = file.split('_')[1]+'_'+sheet)
        main_df = main_df.merge(temp_df,
                                how = 'outer',
                                left_on = ['country','year'],
                                right_on = ['country','year'])

main_df

,country,year,production_total,production_coal,production_natural_gas,production_petrolium_and_other_liquids,production_nuclear,production_nuclear_renewables_and_other,production_renewables_and_other,consumption_total,consumption_coal,consumption_natural_gas,consumption_petrolium_and_other_liquids,consumption_nuclear,consumption_nuclear_renewables_and_other,consumption_renewables_and_other
0,World,1980,296.214353,79.991943,54.761046,133.111109,7.5757,28.350256,20.774555,292.94159,78.657137,53.865223,132.105451,7.5757,28.313779,20.738079
1,Afghanistan,1980,0.072561,0.002355,0.06282,0,NaN,0.007386,0.007386,0.026583,0.002479,0.002094,0.014624,NaN,0.007386,0.007386
2,Albania,1980,0.155562,0.013229,0.01047,0.10154,NaN,0.030323,0.030323,0.162982,0.024317,0.01047,0.099297,NaN,0.028897,0.028897
3,Algeria,1980,2.803017,0.000076,0.48498,2.315385,NaN,0.002576,0.002576,0.780695,0.002547,0.5428,0.232741,NaN,0.002607,0.002607
4,American Samoa,1980,0,0,0,0,NaN,0,0,0.005893,0,0,0.005893,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Wake Island,2019,0,0,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0
9236,Western Sahara,2019,0,0,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0
9237,Yemen,2019,NaN,0,NaN,0.127472,NaN,--,--,NaN,0.003089,NaN,NaN,NaN,NaN,NaN
9238,Zambia,2019,NaN,0.032596,0,0,NaN,--,--,NaN,0.034151,0,NaN,NaN,NaN,NaN


### Energy Import/Export
---
- crude oil including lease condensate import/exports (Mb/d)
- dry natural gas import/exports (bcf)
- coal import/exports (Mst)
- LPG and ethane import/exports (Mb/d)
    - LPG = liquefied petrolium gas, but this was excluded because it had only up to 2014
- electricity import/exports (billion kWh)

In [3]:
# energy import/export data for crude oil, natural gas, coal, lpg/ethane, electricity

sources = ['crude_oil', 'natural_gas', 'coal', 'electricity']

for activity in ['imports','exports']:
    for source in sources:
        
        # reading in each sheet in both files as temporary dataframe
        temp_df = pd.read_csv('../data/'+activity+'_'+source+'.csv')
        
        temp_df = pd.melt(temp_df,
                          id_vars = ['country'],
                          value_vars = temp_df.columns[1:],
                          var_name = ['year'],
                          value_name = activity+'_'+source)
        
        temp_df['year'] = temp_df['year'].astype(int)
        
        Cote_dIvoire = temp_df[temp_df['country'].str.contains('e d')].index
        temp_df.loc[Cote_dIvoire,'country'] = '        Côte d’Ivoire'
        
        main_df = main_df.merge(temp_df,
                                how = 'outer',
                                left_on = ['country','year'],
                                right_on = ['country','year'])
        
main_df

,country,year,production_total,production_coal,production_natural_gas,production_petrolium_and_other_liquids,production_nuclear,production_nuclear_renewables_and_other,production_renewables_and_other,consumption_total,...,consumption_nuclear_renewables_and_other,consumption_renewables_and_other,imports_crude_oil,imports_natural_gas,imports_coal,imports_electricity,exports_crude_oil,exports_natural_gas,exports_coal,exports_electricity
0,World,1980,296.214353,79.991943,54.761046,133.111109,7.5757,28.350256,20.774555,292.94159,...,28.313779,20.738079,29583.029,NaN,284893.825,145.5369836,30580.206,NaN,297478.5297,150.131648
1,Afghanistan,1980,0.072561,0.002355,0.06282,0,NaN,0.007386,0.007386,0.026583,...,0.007386,0.007386,0,NaN,0,0,0,NaN,0,0
2,Albania,1980,0.155562,0.013229,0.01047,0.10154,NaN,0.030323,0.030323,0.162982,...,0.028897,0.028897,0,NaN,176.3696,0,0,NaN,0,0.5
3,Algeria,1980,2.803017,0.000076,0.48498,2.315385,NaN,0.002576,0.002576,0.780695,...,0.002607,0.002607,3.232,NaN,99.06257471,0.07,713.534,NaN,0,0.061
4,American Samoa,1980,0,0,0,0,NaN,0,0,0.005893,...,0,0,0,NaN,0,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Wake Island,2019,0,0,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,0,NaN,0,0,0
9236,Western Sahara,2019,0,0,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,0,NaN,0,0,0
9237,Yemen,2019,NaN,0,NaN,0.127472,NaN,--,--,NaN,...,NaN,NaN,NaN,0,132.2772,0,NaN,0,0,0
9238,Zambia,2019,NaN,0.032596,0,0,NaN,--,--,NaN,...,NaN,NaN,NaN,0,82.22995052,0.198,NaN,0,13.02795718,0.976


### Energy Intensity
---
Defined in two ways:
- consumption per capita (units in million `Btus` per person, or alternatively (MM`Btu`/person))
- consumption per GDP (units in thousand `Btus` per PPP, or alternatively (1000 `Btu`/2015$ GDP PPP)).

Purchasing Power Parity (PPP) to be explained in the next section.

In [4]:
# adding in both definitions of energy intensity into the data frame

int_sheets = ['consumption per capita', 'consumption per GDP']

for sheet in int_sheets: # reading in each sheet as temporary dataframe
    temp_df = pd.read_excel('../data/energy_intensity.xlsx',sheet_name=sheet)

    # melting temp_df and merging into main_df
    temp_df = pd.melt(temp_df,
                    id_vars = ['country'],
                    value_vars = temp_df.columns[1:],
                    var_name = ['year'],
                    value_name = sheet.replace(' ','_'))
    
    main_df = main_df.merge(temp_df,
                            how = 'outer',
                            left_on = ['country','year'],
                            right_on = ['country','year'])
        
main_df

,country,year,production_total,production_coal,production_natural_gas,production_petrolium_and_other_liquids,production_nuclear,production_nuclear_renewables_and_other,production_renewables_and_other,consumption_total,...,imports_crude_oil,imports_natural_gas,imports_coal,imports_electricity,exports_crude_oil,exports_natural_gas,exports_coal,exports_electricity,consumption_per_capita,consumption_per_GDP
0,World,1980,296.214353,79.991943,54.761046,133.111109,7.5757,28.350256,20.774555,292.94159,...,29583.029,NaN,284893.825,145.5369836,30580.206,NaN,297478.5297,150.131648,68.155646,10.558174
1,Afghanistan,1980,0.072561,0.002355,0.06282,0,NaN,0.007386,0.007386,0.026583,...,0,NaN,0,0,0,NaN,0,0,1.990283,0
2,Albania,1980,0.155562,0.013229,0.01047,0.10154,NaN,0.030323,0.030323,0.162982,...,0,NaN,176.3696,0,0,NaN,0,0.5,60.752906,0
3,Algeria,1980,2.803017,0.000076,0.48498,2.315385,NaN,0.002576,0.002576,0.780695,...,3.232,NaN,99.06257471,0.07,713.534,NaN,0,0.061,40.615303,0
4,American Samoa,1980,0,0,0,0,NaN,0,0,0.005893,...,0,NaN,0,0,0,NaN,0,0,180.515604,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Wake Island,2019,0,0,0,0,NaN,0,0,NaN,...,NaN,0,0,0,NaN,0,0,0,NaN,NaN
9236,Western Sahara,2019,0,0,0,0,NaN,0,0,NaN,...,NaN,0,0,0,NaN,0,0,0,NaN,NaN
9237,Yemen,2019,NaN,0,NaN,0.127472,NaN,--,--,NaN,...,NaN,0,132.2772,0,NaN,0,0,0,NaN,NaN
9238,Zambia,2019,NaN,0.032596,0,0,NaN,--,--,NaN,...,NaN,0,82.22995052,0.198,NaN,0,13.02795718,0.976,NaN,NaN


### Purchasing Power Parity
---
Purchasing Power Parity (PPP) refers to the strength of a currency against another. In this dataset, we use 2015 U.S. dollars, in billions, as the baseline to which other currencies are compared. 

For example, in 2015, United States had a PPP of \\$18 trillion, reflecting its actual GDP that year. In the same year, United Kingdom and United Arab Emirates had a PPPs of \\$2 trillion and \\$601 billion.

(For more on purchasing power parity, see [investopedia](https://www.investopedia.com/updates/purchasing-power-parity-ppp/#pairing-purchasing-power-parity-with-gross-domestic-product).)

In [5]:
# melting ppp_df and merging into main_df
ppp = pd.read_excel('../data/purchasing_power_parity.xlsx')

ppp_append = pd.melt(ppp,
                  id_vars = ['country'],
                  value_vars = ppp.columns[1:-1],
                  var_name = ['year'],
                  value_name = 'ppp_2015USD')

main_df = main_df.merge(ppp_append,
                        how = 'outer',
                        left_on = ['country','year'],
                        right_on = ['country','year'])

main_df

,country,year,production_total,production_coal,production_natural_gas,production_petrolium_and_other_liquids,production_nuclear,production_nuclear_renewables_and_other,production_renewables_and_other,consumption_total,...,imports_natural_gas,imports_coal,imports_electricity,exports_crude_oil,exports_natural_gas,exports_coal,exports_electricity,consumption_per_capita,consumption_per_GDP,ppp_2015USD
0,World,1980,296.214353,79.991943,54.761046,133.111109,7.5757,28.350256,20.774555,292.94159,...,NaN,284893.825,145.5369836,30580.206,NaN,297478.5297,150.131648,68.155646,10.558174,27745.479547
1,Afghanistan,1980,0.072561,0.002355,0.06282,0,NaN,0.007386,0.007386,0.026583,...,NaN,0,0,0,NaN,0,0,1.990283,0,NaN
2,Albania,1980,0.155562,0.013229,0.01047,0.10154,NaN,0.030323,0.030323,0.162982,...,NaN,176.3696,0,0,NaN,0,0.5,60.752906,0,NaN
3,Algeria,1980,2.803017,0.000076,0.48498,2.315385,NaN,0.002576,0.002576,0.780695,...,NaN,99.06257471,0.07,713.534,NaN,0,0.061,40.615303,0,NaN
4,American Samoa,1980,0,0,0,0,NaN,0,0,0.005893,...,NaN,0,0,0,NaN,0,0,180.515604,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Wake Island,2019,0,0,0,0,NaN,0,0,NaN,...,0,0,0,NaN,0,0,0,NaN,NaN,NaN
9236,Western Sahara,2019,0,0,0,0,NaN,0,0,NaN,...,0,0,0,NaN,0,0,0,NaN,NaN,NaN
9237,Yemen,2019,NaN,0,NaN,0.127472,NaN,--,--,NaN,...,0,132.2772,0,NaN,0,0,0,NaN,NaN,53.0838
9238,Zambia,2019,NaN,0.032596,0,0,NaN,--,--,NaN,...,0,82.22995052,0.198,NaN,0,13.02795718,0.976,NaN,NaN,247.1819


### Population
---
Population measured in millions.

In [6]:
# melting pop_df and merging into main_df
pop = pd.read_excel('../data/population.xlsx')

pop_append = pd.melt(pop,
                  id_vars = ['country'],
                  value_vars = pop.columns[1:-1],
                  var_name = ['year'],
                  value_name = 'population')

main_df = main_df.merge(pop_append,
                        how = 'outer',
                        left_on = ['country','year'],
                        right_on = ['country','year'])

main_df

,country,year,production_total,production_coal,production_natural_gas,production_petrolium_and_other_liquids,production_nuclear,production_nuclear_renewables_and_other,production_renewables_and_other,consumption_total,...,imports_coal,imports_electricity,exports_crude_oil,exports_natural_gas,exports_coal,exports_electricity,consumption_per_capita,consumption_per_GDP,ppp_2015USD,population
0,World,1980,296.214353,79.991943,54.761046,133.111109,7.5757,28.350256,20.774555,292.94159,...,284893.825,145.5369836,30580.206,NaN,297478.5297,150.131648,68.155646,10.558174,27745.479547,4298126.5224
1,Afghanistan,1980,0.072561,0.002355,0.06282,0,NaN,0.007386,0.007386,0.026583,...,0,0,0,NaN,0,0,1.990283,0,NaN,13356.5
2,Albania,1980,0.155562,0.013229,0.01047,0.10154,NaN,0.030323,0.030323,0.162982,...,176.3696,0,0,NaN,0,0.5,60.752906,0,NaN,2682.7
3,Algeria,1980,2.803017,0.000076,0.48498,2.315385,NaN,0.002576,0.002576,0.780695,...,99.06257471,0.07,713.534,NaN,0,0.061,40.615303,0,NaN,19221.7
4,American Samoa,1980,0,0,0,0,NaN,0,0,0.005893,...,0,0,0,NaN,0,0,180.515604,0,NaN,32.646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Wake Island,2019,0,0,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,0,NaN,NaN,NaN,NaN
9236,Western Sahara,2019,0,0,0,0,NaN,0,0,NaN,...,0,0,NaN,0,0,0,NaN,NaN,NaN,NaN
9237,Yemen,2019,NaN,0,NaN,0.127472,NaN,--,--,NaN,...,132.2772,0,NaN,0,0,0,NaN,NaN,53.0838,29162.2
9238,Zambia,2019,NaN,0.032596,0,0,NaN,--,--,NaN,...,82.22995052,0.198,NaN,0,13.02795718,0.976,NaN,NaN,247.1819,17873.85


In [7]:
#stripping whitespaces out of 'country' and saving as csv
stripped_country = {country:country.strip() for country in main_df['country'].to_list()}
main_df['country'] = main_df['country'].map(stripped_country)
main_df['year'] = main_df['year'].astype(int)
main_df.to_csv('../data/energy_data.csv', index=False)

### Emissions
---
- CO2 emissions (MMtonnes CO2)
- Coal and coke (MMtonnes CO2)
- Consumed natural gas (MMtonnes CO2)
- Petroleum and other liquids (MMtonnes CO2)

In [8]:
# emissions

emissions = ['co2_emissions', 'coal_and_coke', 'consumed_natural_gas', 'petroleum_and_other_liquids']

for emission in emissions:
    temp_df = pd.read_excel('../data/emissions.xlsx', sheet_name = emission)

    temp_df = pd.melt(temp_df,
                      id_vars = ['country'],
                      value_vars = temp_df.columns[1:],
                      var_name = ['year'],
                      value_name = 'emissions'+'_'+emission)

    temp_df['year'] = temp_df['year'].astype(int)
    
    main_df = main_df.merge(temp_df,
                        how = 'outer',
                        left_on = ['country','year'],
                        right_on = ['country','year'])

main_df

,country,year,production_total,production_coal,production_natural_gas,production_petrolium_and_other_liquids,production_nuclear,production_nuclear_renewables_and_other,production_renewables_and_other,consumption_total,...,exports_coal,exports_electricity,consumption_per_capita,consumption_per_GDP,ppp_2015USD,population,emissions_co2_emissions,emissions_coal_and_coke,emissions_consumed_natural_gas,emissions_petroleum_and_other_liquids
0,World,1980,296.214353,79.991943,54.761046,133.111109,7.5757,28.350256,20.774555,292.94159,...,297478.5297,150.131648,68.155646,10.558174,27745.479547,4298126.5224,18671.570672,7455.939754,2843.422967,8372.207951
1,Afghanistan,1980,0.072561,0.002355,0.06282,0,NaN,0.007386,0.007386,0.026583,...,0,0,1.990283,0,NaN,13356.5,1.325965,0.231314,0.111101,0.98355
2,Albania,1980,0.155562,0.013229,0.01047,0.10154,NaN,0.030323,0.030323,0.162982,...,0,0.5,60.752906,0,NaN,2682.7,9.618526,2.379753,0.555503,6.68327
3,Algeria,1980,2.803017,0.000076,0.48498,2.315385,NaN,0.002576,0.002576,0.780695,...,0,0.061,40.615303,0,NaN,19221.7,45.663591,0.237672,28.799159,16.62676
4,American Samoa,1980,0,0,0,0,NaN,0,0,0.005893,...,0,0,180.515604,0,NaN,32.646,0.425071,0,0,0.425071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Wake Island,2019,0,0,0,0,NaN,0,0,NaN,...,0,0,NaN,NaN,NaN,NaN,--,0,0,--
9236,Western Sahara,2019,0,0,0,0,NaN,0,0,NaN,...,0,0,NaN,NaN,NaN,NaN,--,0,0,--
9237,Yemen,2019,NaN,0,NaN,0.127472,NaN,--,--,NaN,...,0,0,NaN,NaN,53.0838,29162.2,--,0.288193,--,--
9238,Zambia,2019,NaN,0.032596,0,0,NaN,--,--,NaN,...,13.02795718,0.976,NaN,NaN,247.1819,17873.85,--,3.186294,0,--


### Life Expectancy
---
Retrieved from the [World Bank](https://data.worldbank.org/indicator/SP.DYN.LE00.IN).

In [9]:
# reading data, making dictionary, creating function to impute life expectancy into main_df
life_exp = pd.read_excel('../data/life_expectancy.xlsx')

# list of countries in main_df not in life_exp
not_in_countries = [country for country in set(main_df['country']) \
                    if country not in set(life_exp['country'])]

life_exp_dict = life_exp.set_index('country').T.to_dict()

def get_life_exp(country, year):
    if country in not_in_countries or int(year) not in range(1980,2020):
        return np.nan
    else:
        return life_exp_dict[country][year]
    
main_df['life_expectancy'] = main_df.apply(lambda x: get_life_exp(x['country'],str(x['year'])),
                                       result_type='expand', axis = 1)

main_df.to_csv('../data/energy_data.csv', index=False)

main_df

,country,year,production_total,production_coal,production_natural_gas,production_petrolium_and_other_liquids,production_nuclear,production_nuclear_renewables_and_other,production_renewables_and_other,consumption_total,...,exports_electricity,consumption_per_capita,consumption_per_GDP,ppp_2015USD,population,emissions_co2_emissions,emissions_coal_and_coke,emissions_consumed_natural_gas,emissions_petroleum_and_other_liquids,life_expectancy
0,World,1980,296.214353,79.991943,54.761046,133.111109,7.5757,28.350256,20.774555,292.94159,...,150.131648,68.155646,10.558174,27745.479547,4298126.5224,18671.570672,7455.939754,2843.422967,8372.207951,62.841745
1,Afghanistan,1980,0.072561,0.002355,0.06282,0,NaN,0.007386,0.007386,0.026583,...,0,1.990283,0,NaN,13356.5,1.325965,0.231314,0.111101,0.98355,43.244000
2,Albania,1980,0.155562,0.013229,0.01047,0.10154,NaN,0.030323,0.030323,0.162982,...,0.5,60.752906,0,NaN,2682.7,9.618526,2.379753,0.555503,6.68327,70.208000
3,Algeria,1980,2.803017,0.000076,0.48498,2.315385,NaN,0.002576,0.002576,0.780695,...,0.061,40.615303,0,NaN,19221.7,45.663591,0.237672,28.799159,16.62676,58.198000
4,American Samoa,1980,0,0,0,0,NaN,0,0,0.005893,...,0,180.515604,0,NaN,32.646,0.425071,0,0,0.425071,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Wake Island,2019,0,0,0,0,NaN,0,0,NaN,...,0,NaN,NaN,NaN,NaN,--,0,0,--,NaN
9236,Western Sahara,2019,0,0,0,0,NaN,0,0,NaN,...,0,NaN,NaN,NaN,NaN,--,0,0,--,NaN
9237,Yemen,2019,NaN,0,NaN,0.127472,NaN,--,--,NaN,...,0,NaN,NaN,53.0838,29162.2,--,0.288193,--,--,66.125000
9238,Zambia,2019,NaN,0.032596,0,0,NaN,--,--,NaN,...,0.976,NaN,NaN,247.1819,17873.85,--,3.186294,0,--,63.886000
